In [1]:
import pandas as pd
import plotly.express as px

In [2]:
pd.read_pickle("failed_experiments.pkl").columns

Index([], dtype='object')

In [3]:
# df = pd.read_pickle("results.pkl")
df = pd.concat([pd.read_pickle("Multiple Pairs Results.pkl"), pd.read_pickle("Single Pairs Results.pkl")])
df["nodes"] = df["nodes"].astype(str)
df["experiment_type"] = df["experiment_type"].map(lambda x: "Single Pairs" if x == "Single" else x)
df["perturber_class"] = df["perturber_class"].map(lambda x: "GreedyMin" if x == "MinFirst" else x)
df.columns

Index(['Original Distance', 'Time Taken', 'Success', 'Number of Paths',
       'Number of Edges', 'Add Times', 'Perturb Times', 'Iterations',
       'Final Distance', 'Status', 'LP Status', 'Perturbation Dict',
       'Total Perturbations', 'Global Budget Slack', 'Supporting Paths',
       'Perturbation Failure', 'graph_name', 'weights', 'experiment_type',
       'n_nodes_per_experiment', 'n_experiments', 'n_trials',
       'min_path_length', 'condition_index', 'use_multithreading', 'nodes',
       'pairs', 'configuration_index', 'perturber_class', 'global_budget',
       'local_budget', 'epsilon', 'k', 'top_k', 'max_iterations', 'max_paths',
       'source', 'target'],
      dtype='object')

In [4]:
print(len(df), sum(df["Success"]), sum(df["Success"])/len(df))
px.histogram(df, x="perturber_class", color="Status",facet_col="experiment_type",hover_data=["LP Status"], category_orders={"perturber_class": ["PathAttack", "GreedyFirst", "GreedyMin"]}).show()

2160 1965 0.9097222222222222


In [5]:
df["Average Add Time"] = df["Add Times"].map(lambda x: sum(x) / len(x) if len(x)>0 else None)
df["Average Perturb Time"] = df["Perturb Times"].map(lambda x: sum(x) / len(x) if len(x)>0 else None)
df["Total Add Time"] = df["Add Times"].map(sum)
df["Total Perturb Time"] = df["Perturb Times"].map(sum)
df["Overhead Time"] = df["Time Taken"] - df["Total Add Time"] - df["Total Perturb Time"]

In [6]:
df.to_csv("results.csv")

In [7]:
index_cols = ["nodes", "condition_index", "configuration_index"]
global_cols = ["epsilon", "n_nodes_per_experiment", "n_experiments", "n_trials", "min_path_length", "use_multithreading", "global_budget", "local_budget"]
extra_cols = ["Status","Add Times", "Perturb Times", "Perturbation Dict","source", "target", "LP Status", "IIS_paths", "IIS_edges", "IIS_global_budget", "Supporting Paths", "Global Budget Slack"]
config_cols = ["perturber_class", "k", "graph_name", "weights", "experiment_type"]
data_cols = ["Time Taken", "Iterations", "Number of Paths", "Number of Edges", "Original Distance", "Final Distance", "Success", "Total Perturbations","Total Add Time", "Total Perturb Time", "Overhead Time"]

In [8]:
# Average Over Trials
a = df[index_cols+data_cols].groupby(index_cols).mean()
a = a.join(df.set_index(index_cols)[config_cols]) # Add back config information
a = a.drop_duplicates()
len(a)

720

In [9]:
## TODO: Instances where the LP is infeasible or runs out of iterations should be removed from the data

for y_val in ["Time Taken", "Total Perturbations", "Number of Paths"]:
    for experiment_type in a["experiment_type"].unique():
        if a.empty: continue
        fig = px.box(a[a["experiment_type"] == experiment_type], 
                    title=f"{experiment_type}: {y_val}",
                    # x="nodes", 
                    y=y_val, 
                    boxmode="group", 
                    color="perturber_class", 
                    facet_col="weights", 
                    facet_row="k",  
                    height=1000,
                    hover_data=data_cols,
                    category_orders={"Status": ["Success"]+[s for s in df["Status"].unique() if s != "Success"],
                                    "Success": [True, False]},
                    points="all",
                    # boxpoints=False,
                    )
        fig.update_xaxes(visible=False, matches=None)
        # fig.update_yaxes(range=[-1,13])
        # fig.update_traces(boxmean=True) 
        # fig.update_xaxes(visible=True)
        fig.show()

In [10]:
b = a.reset_index().set_index(["configuration_index", "condition_index"])

In [11]:

for experiment_type in a.experiment_type.unique():
    fig = px.bar(b[b["experiment_type"]==experiment_type], 
                title=f"{experiment_type} Results",
                x="nodes", 
                y="Time Taken", 
                barmode="group", 
                color="perturber_class", 
                facet_col="weights", 
                facet_row="k", 
                pattern_shape="Success", 
                # height=1000,
                # hover_data=["top_k", "Average Add Time", "Average Perturb Time", "Original Distance", "Number of Paths", "Number of Edges", "Iterations", 'Final Distance',"Status", "LP Status", "nodes"],
                category_orders={"experiment_type": ["Single", "Sets", "Multiple Pairs"], 
                                "Status": ["Success"]+[s for s in df["Status"].unique() if s != "Success"],
                                "Success": [True, False]}
                )
    fig.update_xaxes(visible=False)
    fig.update_xaxes(matches=None)
    # fig.update_xaxes(visible=True)
    fig.show()

In [12]:
len(a)

720

In [13]:
c = a.reset_index().set_index(["condition_index", "nodes"]+config_cols[1:]).groupby("perturber_class")["Total Perturbations"]

In [14]:
config_cols[1:]

['k', 'graph_name', 'weights', 'experiment_type']

In [21]:
optimal = c.get_group("PathAttack").reset_index().set_index(["condition_index", "nodes"]+config_cols[1:])
diffs = pd.DataFrame()
for name, group in c:
    diffs[name] = (group.reset_index().set_index(["condition_index", "nodes"]+config_cols[1:])-optimal)

In [22]:
diffs

GreedyFirst  \
condition_index nodes                                              k graph_name weights experiment_type                
2               ((1069, 100), (2259, 69), (2920, 516), (1716, 1... 3 Facebook   Equal   Multiple Pairs    784.900000   
                                                                   5 Facebook   Equal   Multiple Pairs   1622.900000   
1               ((1288, 556), (3397, 1271), (2310, 2190), (3419... 3 Facebook   Poisson Multiple Pairs   2476.500000   
                                                                   5 Facebook   Poisson Multiple Pairs   5206.500000   
0               ((1341, 502), (2367, 482), (2846, 5), (3210, 11... 3 Facebook   Uniform Multiple Pairs   1401.490478   
...                                                                                                              ...   
1               (3661, 538)                                        5 Facebook   Poisson Single Pairs        3.000000   
2               (3848, 105)                                        3 Facebook   Equal   Single Pairs       52.400000   
                                                                   5 Facebook   Equal   Single Pairs      108.400000   
                (3919, 72)                                         3 Facebook   Equal   Single Pairs      166.100000   
                                                                   5 Facebook   Equal   Single Pairs      342.100000   

                                                                                                           GreedyMin  \
condition_index nodes                                              k graph_name weights experiment_type                
2               ((1069, 100), (2259, 69), (2920, 516), (1716, 1... 3 Facebook   Equal   Multiple Pairs    784.900000   
                                                                   5 Facebook   Equal   Multiple Pairs   1622.900000   
1               ((1288, 556), (3397, 1271), (2310, 2190), (3419... 3 Facebook   Poisson Multiple Pairs   3412.300000   
                                                                   5 Facebook   Poisson Multiple Pairs   7940.300000   
0               ((1341, 502), (2367, 482), (2846, 5), (3210, 11... 3 Facebook   Uniform Multiple Pairs   1927.396337   
...                                                                                                              ...   
1               (3661, 538)                                        5 Facebook   Poisson Single Pairs       97.500000   
2               (3848, 105)                                        3 Facebook   Equal   Single Pairs       52.400000   
                                                                   5 Facebook   Equal   Single Pairs      108.400000   
                (3919, 72)                                         3 Facebook   Equal   Single Pairs      166.100000   
                                                                   5 Facebook   Equal   Single Pairs      342.100000   

                                                                                                         PathAttack  
condition_index nodes                                              k graph_name weights experiment_type              
2               ((1069, 100), (2259, 69), (2920, 516), (1716, 1... 3 Facebook   Equal   Multiple Pairs          0.0  
                                                                   5 Facebook   Equal   Multiple Pairs          0.0  
1               ((1288, 556), (3397, 1271), (2310, 2190), (3419... 3 Facebook   Poisson Multiple Pairs          0.0  
                                                                   5 Facebook   Poisson Multiple Pairs          0.0  
0               ((1341, 502), (2367, 482), (2846, 5), (3210, 11... 3 Facebook   Uniform Multiple Pairs          0.0  
...                                                                                                             ...  
1               (3661, 538)            